In [4]:
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="hi!"),
]

llm.invoke(messages)

AIMessage(content='你好！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5aeff469-58bd-4483-90ec-a63d79bda536-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

In [19]:
import chromadb
from chromadb import Settings

from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from config import CHROMA_DATA_PATH,CHROMA_TENANT, CHROMA_DATABASE

CHROMA_CLIENT = chromadb.PersistentClient(
        path=CHROMA_DATA_PATH,
        settings=Settings(allow_reset=True, anonymized_telemetry=False),
        tenant=CHROMA_TENANT,
        database=CHROMA_DATABASE,
    )

collection_name = "e8b34609-53af-49e0-81ea-552c2e1e81c2"
# Create LangChain Chroma instance
langchain_chroma = Chroma(
    client=CHROMA_CLIENT,
    collection_name=collection_name,
    embedding_function=OpenAIEmbeddings(),  # Replace with your preferred embedding function
)

In [ ]:
documents = langchain_chroma.get(ids=["document_id_1", "document_id_2"])

all_documents = langchain_chroma.get_all()

print(all_documents)

In [15]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# Load, chunk and index the contents of the blog.
loader = TextLoader(r"D:\appChat_demo\backend\data\docs\tesla.txt", autodetect_encoding=True)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("summarize the content in 20 words")

"Company's 2007 1/2 sports car sold out, offering solar-photovoltaic system for home charging, providing 50 miles of travel without grid use."

In [20]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = langchain_chroma.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

query = "summarize the content in 20 words"


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
context = rag_chain.invoke(query)

In [21]:
print(context)

I'm sorry, but there is no provided context to summarize in 20 words.
